In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Merge, Input, concatenate
#from keras.utils.visualize_util import plot, to_graph
#import theano.d3viz as d3v
import word2vec_utils as w2v
import data
import numpy as np
from data_utils import split_dataset 
from chat_constants import *

Using Theano backend.


In [2]:
w2v_model = w2v.initialize()

In [3]:
assert( len(w2v_model.vocab) == 3000000 )


In [4]:
LSTM_DROPOUT = 0.15
A1, B, A2 = data.load_data()
def get_sets_of_data(size=5000):
    A1, B, A2 = data.load_data()
    All_data = np.array([A1,B,A2])
    num_sections = len(A1)//size
    sets = []
    for i in range(num_sections):
        sets.append(All_data[:,i*size:(i+1)*size])
    return(sets)
sets = get_sets_of_data()
adam = keras.optimizers.Adam(lr = 0.025)


In [5]:
### A Layer Pre Train ### 
A_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
A_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_a", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
a_pretrain = Model( inputs = [A_Input], outputs = [ A_layer1(A_Input)])
a_pretrain.load_weights('AmergeB_len30_301dim_w2v_in_and_out.h5',by_name = True)
a_pretrain.compile( optimizer='rmsprop', loss = 'cosine_proximity',metrics = ['accuracy'] )


In [9]:
for i in range(1,3):
    a1,b,a2 = sets[-i]
    A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
    a_pretrain.load_weights('a_pretrain.h5',by_name=True)
    a_pretrain.fit([A1_train],A2_train, batch_size = 200, epochs = 5)
    a_pretrain.save('a_pretrain.h5')

Epoch 1/5
5000/5000 [==============================] - 27s - loss: -0.0021 - acc: 0.4813    
Epoch 2/5
5000/5000 [==============================] - 26s - loss: -0.0021 - acc: 0.4808    
Epoch 3/5
5000/5000 [==============================] - 26s - loss: -0.0021 - acc: 0.4811    
Epoch 4/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4804    
Epoch 5/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4809    
Epoch 1/5
5000/5000 [==============================] - 25s - loss: -0.0021 - acc: 0.4876    
Epoch 2/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4880    
Epoch 3/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4881    
Epoch 4/5
5000/5000 [==============================] - 26s - loss: -0.0021 - acc: 0.4880    
Epoch 5/5
5000/5000 [==============================] - 25s - loss: -0.0021 - acc: 0.4884    


In [10]:
### A Layer Pre Train ### 
B_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
B_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_b", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
b_pretrain = Model( inputs = [B_Input], outputs = [ B_layer1(B_Input)])
#b_pretrain.load_weights('AmergeB_len30_301dim_w2v_in_and_out.h5',by_name = True)
b_pretrain.compile( optimizer='rmsprop', loss = 'cosine_proximity',metrics = ['accuracy'] )


In [11]:
for i in range(1,3):
    a1,b,a2 = sets[-i]
    A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
    a_pretrain.load_weights('a_pretrain.h5',by_name=True)
    a_pretrain.fit([B_train],A2_train, batch_size = 200, epochs = 5)
    a_pretrain.save('a_pretrain.h5')

Epoch 1/5
5000/5000 [==============================] - 26s - loss: -0.0021 - acc: 0.4813    
Epoch 2/5
5000/5000 [==============================] - 22s - loss: -0.0021 - acc: 0.4818    
Epoch 3/5
5000/5000 [==============================] - 21s - loss: -0.0021 - acc: 0.4820    
Epoch 4/5
5000/5000 [==============================] - 24s - loss: -0.0021 - acc: 0.4822    
Epoch 5/5
5000/5000 [==============================] - 24s - loss: -0.0021 - acc: 0.4817    
Epoch 1/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4891    
Epoch 2/5
5000/5000 [==============================] - 23s - loss: -0.0021 - acc: 0.4890    
Epoch 3/5
5000/5000 [==============================] - 24s - loss: -0.0021 - acc: 0.4888    
Epoch 4/5
5000/5000 [==============================] - 25s - loss: -0.0021 - acc: 0.4897    
Epoch 5/5
5000/5000 [==============================] - 25s - loss: -0.0021 - acc: 0.4894    


In [5]:
A_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))
B_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))

A_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_a", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
B_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_b", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A_layer1.trainable = False
B_layer1.trainable = True
merge    = concatenate( [A_layer1(A_Input),B_layer1(B_Input)])

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_ab1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "layer_ab2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer2(AB_layer1(merge))

chat_model = Model(inputs = [ A_Input,B_Input], outputs = [AB_output])

chat_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30, 301)       0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 30, 301)       0                                            
____________________________________________________________________________________________________
layer_a (LSTM)                   (None, 30, 301)       726012                                       
____________________________________________________________________________________________________
layer_b (LSTM)                   (None, 30, 301)       726012                                       
___________________________________________________________________________________________

In [6]:
chat_model.load_weights('AmergeB_len30_301dim_w2v_in_and_out.h5', by_name = True)
#chat_model.load_weights('a_pretrain.h5',by_name=True)
#chat_model.load_weights('b_pretrain.h5',by_name=True)


In [12]:
chat_model.compile( optimizer='rmsprop',loss = 'cosine_proximity',metrics = ['accuracy'])
A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
chat_model.fit([A1_train,B_train], A2_train, batch_size=200, epochs=5, validation_split = 0.05 )
chat_model.save('AmergeB_len30_301dim_w2v_in_and_out.h5')

/usr/local/lib/python2.7/dist-packages/gensim/models/keyedvectors.py:579: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return word in self.vocab


Train on 4750 samples, validate on 250 samples
Epoch 1/5
1200/4750 [======>.......................] - ETA: 74s - loss: -0.0022 - acc: 0.5883

KeyboardInterrupt: 

In [7]:
i = 1
A1_test,B_test,A2_test = w2v.get_training_data(A1[i:i+1],B[i:i+1],A2[i:i+1])
print(A1[i])
print(B[i])
print(A2[i])

predicted = chat_model.predict([A1_test,B_test])
predicted_words = w2v.unvectorize_sentence( predicted[0] )
print( predicted_words )

1 9 1 @ Well , I thought we'd start with pronunciation , if that's okay with you .
2 8 1 @ Not the hacking and gagging and spitting part . Please .
1 9 1 @ Okay . . . then how 'bout we try out some French cuisine . Saturday ? Night ?
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
(301,)
_ _ _ @ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _


In [11]:
a1,b,a2 = sets[1]